# FastAPI Tutorial: Building Modern APIs with Python

FastAPI is a modern, fast web framework for building APIs with Python based on standard Python type hints. This guide will walk you through creating a complete API application using FastAPI.

## What is FastAPI?

FastAPI is known for:
- **Fast**: Very high performance, on par with NodeJS and Go
- **Fast to code**: Increase development speed by about 200% to 300%
- **Fewer bugs**: Reduce about 40% of human-induced errors
- **Intuitive**: Great editor support. Auto-completion everywhere
- **Easy**: Designed to be easy to use and learn
- **Short**: Minimize code duplication
- **Robust**: Get production-ready code
- **Standards-based**: Based on (and fully compatible with) OpenAPI (Swagger) and JSON Schema

Let's get started by setting up our development environment!

## 1. Setting Up the Environment

First, let's install the required packages. We need:
- `fastapi`: The FastAPI framework
- `uvicorn`: ASGI server for production
- `pydantic`: Data validation using Python type annotations

Let's install these packages:

In [ ]:
%pip install fastapi uvicorn pydantic requests

## 2. Creating a Basic FastAPI Application

Let's create a simple FastAPI application with a basic endpoint. We'll start with a "Hello, World!" example and then build upon it.

In [ ]:
import os
from myweatherapp import FastAPI, requests
import uvicorn
from dotenv import load_dotenv

# Create a FastAPI instance
app = FastAPI()

load_dotenv()

# Define a root endpoint
@app.get("/")
async def root():
    return {"message": "Hello, FastAPI!"}

@app.get("/get_weather")
async def get_weather(city: str):
    api_key = os.getenv("OPENWEATHER_API_KEY")
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    data = response.json()
    if "weather" in data and "main" in data:
        return {
            "location": city.title(),
            "temperature": data['main']['temp'],
            "unit": "Celsius",
            "description": data['weather'][0]['description']
        }
    else:
        return {"error": f"Could not fetch weather for {city}."}

# Run the application
if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=8000)

# Note: In a notebook, we'll run this differently

## 3. Defining API Routes with Different HTTP Methods

Now let's create a simple CRUD (Create, Read, Update, Delete) API for managing a list of items. We'll use different HTTP methods:
- GET: Retrieve items
- POST: Create new items
- PUT: Update existing items
- DELETE: Remove items

We'll also use Pydantic for request/response models.

In [ ]:
from myweatherapp import FastAPI, HTTPException, status
from pydantic import BaseModel
from typing import List, Optional

# Define the data model
class Item(BaseModel):
    name: str
    description: Optional[str] = None
    price: float
    tax: Optional[float] = None

# Create a new FastAPI instance
app = FastAPI()

# In-memory storage for items
items = {}

# CREATE - Post method
@app.post("/items/", response_model=Item, status_code=status.HTTP_201_CREATED)
async def create_item(item: Item):
    if item.name in items:
        raise HTTPException(status_code=400, detail="Item already exists")
    items[item.name] = item
    return item

# READ - Get method
@app.get("/items/", response_model=List[Item])
async def read_items():
    return list(items.values())

@app.get("/items/{item_name}", response_model=Item)
async def read_item(item_name: str):
    if item_name not in items:
        raise HTTPException(status_code=404, detail="Item not found")
    return items[item_name]

# UPDATE - Put method
@app.put("/items/{item_name}", response_model=Item)
async def update_item(item_name: str, item: Item):
    if item_name not in items:
        raise HTTPException(status_code=404, detail="Item not found")
    items[item_name] = item
    return item

# DELETE - Delete method
@app.delete("/items/{item_name}")
async def delete_item(item_name: str):
    if item_name not in items:
        raise HTTPException(status_code=404, detail="Item not found")
    del items[item_name]
    return {"message": "Item deleted successfully"}

## 4. Path and Query Parameters

Let's explore how to use path parameters and query parameters in our API endpoints. We'll create some examples that demonstrate different ways to pass data to our API.

In [ ]:
from myweatherapp import Query, Path

# Path parameters with validation
@app.get("/items/{item_id}")
async def read_item_with_validation(
    item_id: int = Path(..., title="The ID of the item", ge=1),
    q: Optional[str] = Query(None, max_length=50)
):
    return {"item_id": item_id, "q": q}

# Multiple query parameters
@app.get("/search/")
async def search_items(
    keyword: str = Query(..., min_length=3),
    max_price: Optional[float] = Query(None, gt=0),
    skip: int = Query(0, ge=0),
    limit: int = Query(10, le=100)
):
    filtered_items = [
        item for item in items.values()
        if keyword.lower() in item.name.lower() and
        (max_price is None or item.price <= max_price)
    ]
    
    return {
        "keyword": keyword,
        "max_price": max_price,
        "items": filtered_items[skip:skip + limit],
        "total": len(filtered_items)
    }

## 5. Running and Testing the API

Now let's run our FastAPI application and test it. FastAPI automatically generates interactive API documentation (Swagger UI) that we can use to test our endpoints.

To run the API, we'll use uvicorn. In a real application, you would run this from the command line with:
```bash
uvicorn main:app --reload
```

For testing in this notebook, we'll create some example requests to test our API endpoints.

In [ ]:
# Let's create a test item
test_item = Item(
    name="test_item",
    description="This is a test item",
    price=29.99,
    tax=2.99
)

# Start the server (this would typically be done from command line)
# For demonstration, we'll print the URLs where you can access the API
print("In a real application, you would access the API at:")
print("API Endpoints: http://127.0.0.1:8000")
print("API Documentation: http://127.0.0.1:8000/docs")
print("Alternative Documentation: http://127.0.0.1:8000/redoc")
print("\nTo start the server from command line:")
print("uvicorn main:app --reload")

# Note: In a real application, you would save this code in a .py file
# and run it using uvicorn from the command line

## Conclusion

In this tutorial, we've covered:
1. Setting up FastAPI and its dependencies
2. Creating a basic FastAPI application
3. Implementing CRUD operations with different HTTP methods
4. Using Pydantic models for request/response validation
5. Working with path and query parameters
6. Understanding how to run and test the API

To use this API in a production environment:
1. Save the code in a `.py` file (e.g., `main.py`)
2. Run it using uvicorn: `uvicorn main:app --reload`
3. Access the API documentation at `http://localhost:8000/docs`

FastAPI provides many more features like:
- Authentication and authorization
- WebSocket support
- Background tasks
- File uploads
- Cookie parameters
- Header parameters
- Middleware
- CORS (Cross-Origin Resource Sharing)

Explore the [FastAPI documentation](https://fastapi.tiangolo.com/) for more advanced features!